In [4]:
from numpy import *
# import cv2 as cv
from time import sleep
import os
import errno
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.cross_validation import train_test_split

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.utils import np_utils
from keras.layers import Conv2D, MaxPooling2D
import pickle


/anaconda3/lib/python3.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


In [5]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [6]:
def read_data(folder):
    x_data_temp = []
    y_data_temp = []
    x_test_data_temp = []
    y_test_data_temp = []
    
    for file in os.listdir(folder):
        if file.endswith(".meta") or file.endswith(".html"):
            print("Ignoring html and meta files")
        elif "test_batch" in file:
            # test data file detected. we are gonna load it separately
            test_data_temp = unpickle(folder + "/" + file)
            x_test_data_temp.append(test_data_temp[b'data'])
            y_test_data_temp.append(test_data_temp[b'labels'])
        else:
            temp_data = unpickle(folder + "/" + file)
            x_data_temp.append(temp_data[b'data'])
            y_data_temp.append(temp_data[b'labels'])
    x_data = array(x_data_temp)
    y_data = array(y_data_temp)
    x_test_data = array(x_test_data_temp)
    y_test_data = array(y_test_data_temp)
    return [x_data, y_data, x_test_data, y_test_data]


In [7]:
cd /Users/sonalisreedhar/Documents/DIP/HW5

/Users/sonalisreedhar/Documents/DIP/HW5


In [8]:
X_train_temp, y_train_temp, X_test_temp, y_test_temp = read_data("cifar-10-batches-py")

Ignoring html and meta files
Ignoring html and meta files


In [9]:
X_train_temp = X_train_temp.reshape(X_train_temp.shape[0] * X_train_temp.shape[1], X_train_temp.shape[2])
y_train_temp = y_train_temp.reshape(y_train_temp.shape[0] * y_train_temp.shape[1])

# Similarly for X_test_temp and y_test_data

X_test_temp = X_test_temp.reshape(X_test_temp.shape[0] * X_test_temp.shape[1], X_test_temp.shape[2])
y_test_temp = y_test_temp.reshape(y_test_temp.shape[0] * y_test_temp.shape[1])

print(X_train_temp.shape, X_train_temp.ndim, type(X_train_temp))
print(y_train_temp.shape, y_train_temp.ndim, type(y_train_temp))

print(X_test_temp.shape, X_test_temp.ndim, type(X_test_temp))
print(y_test_temp.shape, y_test_temp.ndim, type(y_test_temp))


(50000, 3072) 2 <class 'numpy.ndarray'>
(50000,) 1 <class 'numpy.ndarray'>
(10000, 3072) 2 <class 'numpy.ndarray'>
(10000,) 1 <class 'numpy.ndarray'>


In [10]:

print(X_test_temp.shape, X_test_temp.ndim, type(X_test_temp))
print(y_test_temp.shape, y_test_temp.ndim, type(y_test_temp))

(10000, 3072) 2 <class 'numpy.ndarray'>
(10000,) 1 <class 'numpy.ndarray'>


In [11]:
X_train, y_train = shuffle(X_train_temp, y_train_temp, random_state=4)
X_test, y_test = shuffle(X_test_temp, y_test_temp, random_state=4)

In [12]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=4)

In [13]:
X_train = X_train.reshape(X_train.shape[0],  32, 32,3)
y_train = np_utils.to_categorical(y_train, 10)

In [14]:
X_val = X_val.reshape(X_val.shape[0],  32, 32,3)
y_val = np_utils.to_categorical(y_val, 10)

In [15]:
X_test = X_test.reshape(X_test.shape[0],  32, 32,3)
y_test = np_utils.to_categorical(y_test, 10)

In [16]:
print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)
print(X_test.shape, y_test.shape)

(40000, 32, 32, 3) (40000, 10)
(10000, 32, 32, 3) (10000, 10)
(10000, 32, 32, 3) (10000, 10)


In [17]:
X_train = X_train.astype('float32')
X_val = X_val.astype('float32')
X_test = X_test.astype('float32')

In [18]:
X_train /= 255
X_val /= 255
X_test /= 255

In [21]:
from keras import optimizers
cnnModel = Sequential()
cnnModel.add(Conv2D(6, (5, 5), padding='valid', activation = 'relu', kernel_initializer='he_normal', input_shape=(32,32,3)))
cnnModel.add(MaxPooling2D((2, 2), strides=(1, 1)))
cnnModel.add(Conv2D(16, (5, 5), padding='valid', activation = 'relu', kernel_initializer='he_normal'))
cnnModel.add(MaxPooling2D((2, 2), strides=(1, 1)))
cnnModel.add(Flatten())
cnnModel.add(Dense(120, activation = 'relu', kernel_initializer='he_normal'))
cnnModel.add(Dense(84, activation = 'relu', kernel_initializer='he_normal'))
cnnModel.add(Dense(10, activation = 'softmax', kernel_initializer='he_normal'))
sgd = optimizers.SGD(lr=.1, momentum=0.9, nesterov=True)
cnnModel.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [20]:
# mport keras
from keras import optimizers
# from keras.datasets import cifar10
# from keras.models import Sequential
# from keras.layers import Conv2D, Dense, Flatten, MaxPooling2D
from keras.callbacks import LearningRateScheduler, TensorBoard

In [22]:
import tensorflow
import logging
logging.getLogger('tensorflow').setLevel(logging.ERROR)

In [23]:
import os

os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [27]:
history = cnnModel.fit(X_train, y_train, batch_size=32, epochs=50,
                    verbose=1, validation_data=(X_test, y_test))

Train on 40000 samples, validate on 10000 samples
Epoch 1/50
40000/40000 [==============================] - 90s 2ms/step - loss: 0.2564 - acc: 0.9094 - val_loss: 3.1988 - val_acc: 0.4828
Epoch 2/50
40000/40000 [==============================] - 87s 2ms/step - loss: 0.2333 - acc: 0.9188 - val_loss: 3.2417 - val_acc: 0.4762
Epoch 3/50
40000/40000 [==============================] - 88s 2ms/step - loss: 0.2197 - acc: 0.9223 - val_loss: 3.5468 - val_acc: 0.4690
Epoch 4/50
40000/40000 [==============================] - 88s 2ms/step - loss: 0.2181 - acc: 0.9231 - val_loss: 3.4919 - val_acc: 0.4569
Epoch 5/50
40000/40000 [==============================] - 91s 2ms/step - loss: 0.1986 - acc: 0.9326 - val_loss: 3.7792 - val_acc: 0.4623
Epoch 6/50
40000/40000 [==============================] - 87s 2ms/step - loss: 0.2028 - acc: 0.9288 - val_loss: 3.6997 - val_acc: 0.4627
Epoch 7/50
40000/40000 [==============================] - 87s 2ms/step - loss: 0.1856 - acc: 0.9365 - val_loss: 3.8831 - val_acc

In [30]:
score = cnnModel.evaluate(X_test, y_test, verbose=1)
print("Test Score :", score[0])
print("Test accuracy: ", score[1])
print(cnnModel.predict_classes(X_test[1:50]))
print(y_test[1:50])

10000/10000 [==============================] - 8s 757us/step
Test Score : 4.977294649505615
Test accuracy:  0.4491
[5 0 7 6 9 2 3 7 8 0 5 6 9 3 4 0 3 0 2 6 5 4 1 3 3 4 0 3 7 5 1 1 5 4 6 8 9
 7 9 7 6 7 9 1 2 1 0 7 5]
[[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0. 0. 0.

In [53]:
# set callback
tb_cb = TensorBoard(log_dir='./lenet', histogram_freq=0)
change_lr = LearningRateScheduler(scheduler)
cbks = [change_lr,tb_cb]

In [54]:
 # start train
classifier.fit(X_train, y_train,
              batch_size=128,
              epochs=200,
              callbacks=cbks,
              validation_data=(X_val, y_val),
              shuffle=True)

ValueError: Error when checking input: expected conv2d_3_input to have shape (32, 32, 3) but got array with shape (3, 32, 32)